# Voting

In [21]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

f_size = 16
# Importing the dataset

dataset = pd.read_csv('CVD dataset2.csv')
# dataset = pd.read_csv('CVD-dataset-2-3-11-13-15.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, f_size].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
17


In [22]:
dataNoCvd = data[~(data[:,f_size] > 0.0)]
dataCvd = data[~(data[:,f_size] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [23]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [24]:
def create_train_test_sets(f_size,cv,dataNoCvd,dataCvd,val_ratio):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    validation_total = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
#     print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
#     print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)
    
    Cvd_val = round(504*val_ratio*(rowsCvd/len(data)))
    noCvd_val = round(504*val_ratio)-Cvd_val
#     print(noCvd_val)
    
    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
        X_val = []
        y_val = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)
#         print(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        testSubset_total = find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd)
        if i!=9:
#             print(UpNoCvd,UpNoCvd+noCvd_val,UpCvd,UpCvd+Cvd_val)
            validationSubset_total = find_testValSubset(UpNoCvd,UpNoCvd+noCvd_val,UpCvd,UpCvd+Cvd_val,dataNoCvd,dataCvd)
#             print(DownNoCvd,UpNoCvd+noCvd_val,DownCvd,UpCvd+Cvd_val)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd+noCvd_val,DownCvd,UpCvd+Cvd_val,dataNoCvd,dataCvd)
        else:
#             print(DownNoCvd-noCvd_val,DownNoCvd,DownCvd-Cvd_val,DownCvd)
            validationSubset_total = find_testValSubset(DownNoCvd-noCvd_val,DownNoCvd,DownCvd-Cvd_val,DownCvd,dataNoCvd,dataCvd)
#             print(DownNoCvd-noCvd_val,UpNoCvd,DownCvd-Cvd_val,UpCvd)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd-noCvd_val,UpNoCvd,DownCvd-Cvd_val,UpCvd,dataNoCvd,dataCvd)
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, f_size, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, f_size), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
        
        X_val.append(np.delete(validationSubset_total, f_size, axis=1))
        y_val_temp = np.delete(validationSubset_total, slice(0, f_size), axis=1)
        y_val.append(np.reshape(y_val_temp, len(y_val_temp)))
        X_val_temp = np.array(X_val)
        X_val = X_val_temp[0]
        y_val_temp = np.array(y_val)
        y_val = y_val_temp[0]
        
        X_train.append(np.delete(trainSubset_total, f_size, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, f_size), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        validation_total.append([X_val,y_val])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd

In [28]:
train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(f_size,10,dataNoCvd,dataCvd,0.1)

In [29]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)

454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
454
ratio in train set: 0.07048458149779736
ratio in test set: 0.08928571428571429


In [30]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from statistics import mean
import pandas as pd, numpy as np
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials 

total = []

def objective(space):

    clf = xgb.XGBClassifier(n_estimators = 10000,
                            learning_rate = space['learning_rate'],
                            max_depth = int(space['max_depth']),
                            min_child_weight = int(space['min_child_weight']),
                            gamma = space['gamma'],
                            colsample_bylevel = space['colsample_bylevel'],
                            reg_lambda = space['reg_lambda'],
                            scale_pos_weight = space['scale_pos_weight'],
                            colsample_bytree = space['colsample_bytree'],
                            reg_alpha = space['reg_alpha'],
                            subsample = space['subsample'])


    accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd,clf)

#     print("accuraccy:    ", mean(accuracy))
#     print("specificity:  ", mean(specificity))
#     print("sensitivity:  ", mean(sensitivity))
#     print("auc        :  ", mean(auc))
    total.append([mean(accuracy),mean(specificity),mean(sensitivity),mean(auc)])
#     return np.mean([roc_auc_score(X, y),roc_auc_score(X, y)])
#     return np.mean(cross_val_score(clf, X, y, cv=3, scoring='roc_auc'))
    return{'loss':1-mean(auc), 'status': STATUS_OK , 'auc':mean(auc), 'accuracy':mean(accuracy)}
    




In [33]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd,clf):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        

        X_train = train_total[i][0]
        y_train = train_total[i][1]
        
        X_val = validation_total[i][0]
        y_val = validation_total[i][1]
        
        eval_set  = [(X_train,y_train), (X_val,y_val)]
        
        clf.fit( X_train, y_train, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=50,verbose = False )

        
        
        y_pred = clf.predict(X_test)
        
        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [36]:
# https://gist.github.com/walterreade/6e20dba959277bd9af77

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials 

space ={
        'learning_rate':hp.uniform ('learning_rate', 0.1, 0.5),
        'max_depth': hp.quniform("max_depth", 1, 5, 1),
        'min_child_weight': hp.quniform('min_child_weight', 1, 7, 1),
#         'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.uniform('gamma', 0, 1),
        'colsample_bylevel':hp.uniform('colsample_bylevel', 0.5, 1),
        'reg_lambda':hp.uniform('reg_lambda', 0, 3),
        'scale_pos_weight':hp.quniform("scale_pos_weight", 1, 9, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
        'reg_alpha':hp.uniform('reg_alpha', 0, 0.3),
        'subsample': hp.uniform ('subsample', 0.8, 1)
    }      


# parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
# accuracy : 0.6910714285714286
# spes     : 0.68789592760181
# sens     : 0.74
# auc      : 0.713947963800905
# [13, 15, 11, 2, 3]
# [13, 15, 11, 2, 3, 1, 14, 12, 10, 5, 6, 8, 4, 9, 0, 7, 16]
    
#         }
# maxdepth': hp.choice("xmax_depth",np.arange(5, 30, dtype=int) ),

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=150,
            trials=trials)

print(best)

100%|██████████| 150/150 [01:02<00:00,  2.41trial/s, best loss: 0.41405731523378586]
{'colsample_bylevel': 0.7508535005823251, 'colsample_bytree': 0.7083398153035388, 'gamma': 0.7928795745742508, 'learning_rate': 0.2849656341201214, 'max_depth': 5.0, 'min_child_weight': 5.0, 'reg_alpha': 0.13054827129081417, 'reg_lambda': 1.9213153628660709, 'scale_pos_weight': 7.0, 'subsample': 0.8707849783536298}


In [19]:
print(total)

[[0.5910714285714286, 0.5955505279034691, 0.54, 0.5677752639517346], [0.5982142857142857, 0.6187405731523379, 0.335, 0.4768702865761689], [0.4142857142857143, 0.393552036199095, 0.69, 0.5417760180995475], [0.5660714285714286, 0.5703996983408748, 0.515, 0.5426998491704373], [0.5660714285714286, 0.564630467571644, 0.585, 0.574815233785822], [0.5910714285714286, 0.6012066365007541, 0.46, 0.5306033182503771], [0.5642857142857143, 0.5512443438914028, 0.74, 0.6456221719457014], [0.5160714285714286, 0.5069758672699849, 0.64, 0.5734879336349924], [0.3892857142857143, 0.37051282051282053, 0.64, 0.5052564102564102], [0.625, 0.6340874811463046, 0.515, 0.5745437405731523], [0.475, 0.455052790346908, 0.735, 0.595026395173454], [0.48928571428571427, 0.4800904977375566, 0.615, 0.5475452488687782], [0.44285714285714284, 0.4203996983408748, 0.74, 0.5801998491704374], [0.5964285714285714, 0.6128205128205129, 0.39, 0.5014102564102564], [0.5857142857142857, 0.593552036199095, 0.49, 0.5417760180995475], [0

In [ ]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100,200,300,400],
                    "min_child_weight" :[1,2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3,4,5],
                    "colsample_bytree" :[0.5,0.75],
                    "reg_alpha"        :[0.05,0.1,0.2],
                    "subsample"        :[0.8]
    
        }

In [143]:
params = [[],[],[],[],[],[]]
for i in range(6):
    best_par = optimizer.max.get('params')
    params.append(best_par.get('learning_rate'))
    params.append(round(best_par.get('max_depth')))
    params.append(round(best_par.get('n_estimators')))
    params.append(round(best_par.get('min_child_weight')))
    params.append(best_par.get('gamma'))
    params.append(best_par.get('colsample_bylevel'))
    params.append(best_par.get('reg_lambda'))
    params.append(round(best_par.get('scale_pos_weight')))
    params.append(best_par.get('colsample_bytree'))
    params.append(best_par.get('reg_alpha'))
    params.append(best_par.get('subsample'))

print(o)

3.740878001587038
